# CF for item

In [1]:
import numpy as np
import numpy as np
import time
import scipy.sparse
from scipy.sparse import lil_matrix

### Load data

In [2]:
# load train data
f = open("data/processed_mean_data.dat", 'r')
user_mean_train = dict()
user_max_train = dict()
user_min_train = dict()
user_total_train = dict()
train_set = dict()
song_to_user_dict = dict()

for line in f:
    user, mean, max_value, min_value, total_value = line.strip('\n').split(' ')
    user_mean_train[int(user)] = float(mean)
    user_max_train[int(user)] = int(max_value)
    user_min_train[int(user)] = int(min_value)
    user_total_train[int(user)] = int(total_value)

In [3]:

user_count = 110000
song_count = 386214
user_to_song =lil_matrix((song_count, user_count))
#user_to_song = np.ones((user_count, song_count))
print("Get Song item and its listener.\nTotal User = %d, Total Song = %d" % (user_count, song_count))

f = open("data/processed_data.dat", 'r')
for line in f:
    user, song, times = line.strip("\n").split(' ')
    user_to_song[int(song), int(user)] = 1
    user = int(user)
    song = int(song)
    if user not in train_set.keys():
        train_set[user] = set([int(song)])
    else:
        train_set[user].add(int(song))
    if song not in song_to_user_dict.keys():
        song_to_user_dict[song] = set([user])
    else:
        song_to_user_dict[song].add(user)
print("Finish Getting All Song and its record")



Get Song item and its listener.
Total User = 110000, Total Song = 386214
Finish Getting All Song and its record


In [4]:
# load mean data
f = open("data/result_mean_data.dat", 'r')
user_mean_result = dict()
user_max_result = dict()
user_min_result = dict()
user_total_result = dict()
result_set = dict()

for line in f:
    user, mean, max_value, min_value, total_value = line.strip('\n').split(' ')
    user_mean_result[int(user)] = float(mean)
    user_max_result[int(user)] = int(max_value)
    user_min_result[int(user)] = int(min_value)
    user_total_result[int(user)] = int(total_value)

f = open("data/result_data.dat", 'r')
for line in f:
    user, song, time = line.strip("\n").split(' ')
    user = int(user)
    if user not in result_set.keys():
        result_set[user] = set([int(song)])
    else:
        result_set[user].add(int(song))

### Train Data for previous 1000 user ( array length )

In [48]:
import math
def CalSim(setA, setB):
    dot = float(len(setA & setB))
    if dot == 0:
        return 0
    lenA = float(math.pow(len(setA), 0.5))
    lenB = float(math.pow(len(setB), 0.5))
    return dot*1.0 /(lenA * lenB)

In [103]:
user = 0
total_score = 0 
pre_time = time.clock()
while (user < 100):
    print("User : %d" % user)
    train_index = train_set[user]
    SongScore = np.zeros(song_count, dtype = "float32")
    for song in train_index:
        setA = song_to_user_dict[song]
        for s in range(song_count):
            if s in train_index:
                continue
            if s not in song_to_user_dict.keys():
                continue
            setB = song_to_user_dict[s]
            SimilarityScore = CalSim(setA, setB)
            SongScore[s] += SimilarityScore
    
    indices = np.argsort(SongScore)
    
    correct = 0
    score = 0
    for i in range(100):
        if indices[song_count -1 -i] in result_set[user]:
            print("%d %d %5f" % (i+1, indices[song_count -1 -i], SongScore[indices[song_count -1 -i]]))
            correct += 1
            score += correct / (i+1) / len(result_set[user])
    print("%d/%d" % (correct,len(result_set[user])))
    print("%5f" % score)
    total_score += score
    user += 1

print(total_score)
print(time.clock() - pre_time)

User : 0
1 12985 0.793141
7 127276 0.486848
15 82113 0.346328
21 233124 0.327441
36 148050 0.261629
5/7
0.259297
0.259297052154195
5.979172999999946


### Train Data for previous 1000 user ( matrix )

In [5]:
import math
normArray = np.zeros(song_count, dtype = "float32")
for song in range(song_count):
    score = 0
    ul = user_to_song[song].data[0]
    normArray[song] = math.sqrt(len(ul))
    #for dat in ul:
    #    score += dat * dat
    #score = math.sqrt(score)
    #

In [6]:
import time

In [9]:
pre_time = time.clock()
UserGroup = 0
GroupSize = 100
total_score = 0
while (UserGroup < 30):
    print("UserGroup : %d" % UserGroup)
    
    # initialize a (nUser * len) * 110000 matrix
    firstUser = GroupSize * UserGroup
    lengthMat = 0
    for user in range(GroupSize):
        lengthMat += len(train_set[user + firstUser])
    MatA = np.zeros( (lengthMat, user_count) )
    print(MatA.shape)
    
    ptr = 0
    ptrList = [0]
    songNorm = np.mat(np.eye(lengthMat))
    for user in range(firstUser, firstUser + GroupSize):
        train_index = train_set[user]
        for ind in train_index:
            songNorm[ptr, ptr] = 1 / normArray[ind]
            rowIndex = user_to_song[ind].rows[0]
            for rowNum in rowIndex:
                MatA[ptr, rowNum] = user_to_song[ind, rowNum]
            ptr += 1
        ptrList.append(ptr)
    
    # (nUser*len) * 110000   *    110000 * song_count
    resultMatrix = MatA * user_to_song.transpose()
        
    # row divide for the len of train_index
    resultMatrix = songNorm * resultMatrix
    # print(resultMatrix.shape)
    
    # do sum operation at the axis of y
    ptr = 0
    for user in range(firstUser, firstUser + GroupSize):
        resultUserMatrix = np.sum(resultMatrix[ptrList[ptr]:ptrList[ptr + 1], :], axis = 0)
        resultArray = resultUserMatrix / normArray
        
        for i in range(song_count):
            if math.isnan(resultArray[0, i]):
                resultArray[0, i] = 0
        indices = np.argsort(resultArray)
    
        score = correct = 0
        i = 0
        song_ptr = 1
        while song_ptr <= 100:
            if indices[0, song_count -1 -i] in train_set[user]:
                i += 1
                continue
            if indices[0, song_count -1 -i] in result_set[user]:
                # print("rank: %d indice: %d value:%5f" % (song_ptr, indices[0, song_count -1 -i], resultArray[0, indices[0, song_count -1 -i]] ))
                correct += 1
                score += correct / (song_ptr) / len(result_set[user])
            i += 1
            song_ptr += 1
                
        total_score += score
        #print("user %d: %d / %d" % (user, correct, len(result_set[user])))
        #print(score)
        ptr += 1
        
    UserGroup += 1
    print("Group %d time: %5f" %(UserGroup, time.clock()-pre_time))
    

print("Map: %5f" % (total_score / 3000))

UserGroup : 0
(1294, 110000)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


Group 1 time: 124.577751
UserGroup : 1
(1286, 110000)
Group 2 time: 243.841756
UserGroup : 2
(1322, 110000)
Group 3 time: 364.825268
UserGroup : 3
(1394, 110000)
Group 4 time: 494.772429
UserGroup : 4
(1294, 110000)
Group 5 time: 612.673405
UserGroup : 5
(1326, 110000)
Group 6 time: 735.937015
UserGroup : 6
(1390, 110000)
Group 7 time: 866.109443
UserGroup : 7
(1221, 110000)
Group 8 time: 980.736780
UserGroup : 8
(1418, 110000)
Group 9 time: 1121.771680
UserGroup : 9
(1305, 110000)
Group 10 time: 1255.142049
UserGroup : 10
(1336, 110000)
Group 11 time: 1407.700323
UserGroup : 11
(1208, 110000)
Group 12 time: 1538.019258
UserGroup : 12
(1324, 110000)
Group 13 time: 1673.230312
UserGroup : 13
(1402, 110000)
Group 14 time: 1809.414684
UserGroup : 14
(1239, 110000)
Group 15 time: 1929.544312
UserGroup : 15
(1308, 110000)
Group 16 time: 2055.557348
UserGroup : 16
(1138, 110000)
Group 17 time: 2163.997609
UserGroup : 17
(1385, 110000)
Group 18 time: 2297.138042
UserGroup : 18
(1289, 110000)


In [8]:
print("Map: %5f" % (total_score / 1000))

Map: 0.079196


In [97]:
pre_time = time.clock()
user = 0
while (user < 10):
    print("User : %d" % user)
    resultScore = np.zeros(song_count)
    
    # initialize a len * 110000 matrix
    train_index = train_set[user]
    MatA = np.zeros((len(train_index), user_count))
    ptr = 0
    for ind in train_index:
        rowIndex = user_to_song[ind].rows[0]
        for rowNum in rowIndex:
            MatA[ptr, rowNum] = user_to_song[ind, rowNum]
        ptr += 1
    print(MatA)
    
    # len * 110000   *    110000 * song_count
    print(MatA.shape)
    print(user_to_song.shape)
    resultMatrix = MatA * user_to_song.transpose()
        
    # row divide for the len of train_index
    songNorm = np.mat(np.eye(len(train_index)))
    ptr = 0
    for song in train_index:
        songNorm[ptr, ptr] = 1 / normArray[song] 
        ptr += 1
    print(songNorm)
    resultMatrix = songNorm * resultMatrix
    
    # do sum operation at the axis of y
    resultMatrix = np.sum(resultMatrix, axis = 0)
    resultArray = resultMatrix / normArray
    
    for i in range(song_count):
        if math.isnan(resultArray[0, i]):
            resultArray[0, i] = 0
    indices = np.argsort(resultArray, axis = 1)
    
    correct = 0
    for i in range(100):
        if indices[0,song_count -1 -i] in result_set[user]:
            print("%d %5f" % (indices[ 0,song_count -1 -i], resultArray[0, indices[ 0,song_count -1 -i]] ))
            correct += 1
     
    print("user %d: %d / %d" % (user, correct, len(result_set[user])))
        
    user += 1
    
print(time.clock()-pre_time)


User : 0
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]
(6, 110000)
(386214, 110000)


KeyboardInterrupt: 

In [284]:
result_set[0]

{'127276', '12985', '148050', '233124', '297810', 385877, '82113'}

In [167]:
for i in range(10):
    print("rank: %d , song: %d" % (i, indices[song_count - 1 - i]))

rank: 0 , song: 311120
rank: 1 , song: 311991
rank: 2 , song: 24373
rank: 3 , song: 213025
rank: 4 , song: 133366
rank: 5 , song: 140425
rank: 6 , song: 324741
rank: 7 , song: 62868
rank: 8 , song: 9559
rank: 9 , song: 57353


In [135]:
def CalSim(vecA, vecB, x, y):
    ptrA = ptrB = 0
    score = 0
    while ptrA < len(vecA) and ptrB < len(vecB):
        if vecA[ptrA] == vecB[ptrB]:
            score += user_to_song[x, vecA[ptrA]] * user_to_song[y, vecB[ptrB]]
            ptrA += 1
            ptrB += 1
        elif vecA[ptrA] < vecB[ptrB]:
            ptrA += 1
        else :
            ptrB += 1
    return score / normArray[x] / normArray[y]

In [59]:
user = 1
train_index = train_set[user]
MatA = np.zeros((len(train_index), user_count))
ptr = 0
for ind in train_index:
    rowIndex = user_to_song[ind].rows[0]
    for rowNum in rowIndex:
        MatA[ptr, rowNum] = user_to_song[ind, rowNum]
    ptr += 1